# BMW Project Scouting

In [1]:
from bmwprophet.loader import Loader

In [2]:
loader = Loader()
loader.load_data("data", "Dry_room_Last_month.csv")

In [3]:
loader.print_column_info()

[ ]: Column 'trigger_ts_utc' has 42827 unique values
[ ]: Column 'measured_value' has 86738 unique values
[!]: Column 'ns' has 1 unique values
[!]: Column 'machine' has 1 unique values
[ ]: Column 'errection_location' has 4 unique values
[ ]: Column 'installation_location' has 10 unique values
[ ]: Column 'components' has 17 unique values
[ ]: Column 'sensor_type' has 7 unique values


In [4]:
loader.save_data(path="data", file="preprocessed_data.csv")